In [1]:
import pandas as pd
import requests
#!pip install yfinance
import yfinance as yf

In [2]:
# Завантаження даних Fear and Greed Index API та перетворення на DataFrame
url = requests.get('https://api.alternative.me/fng/?limit=0')
data = url.json()['data']
df = pd.DataFrame(data)
df.head()

,value,value_classification,timestamp,time_until_update
0,9,Extreme Fear,1771459200,17634
1,8,Extreme Fear,1771372800,NaN
2,10,Extreme Fear,1771286400,NaN
3,12,Extreme Fear,1771200000,NaN
4,8,Extreme Fear,1771113600,NaN


In [3]:
# Перевірка типів даних
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2937 entries, 0 to 2936
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   value                 2937 non-null   object
 1   value_classification  2937 non-null   object
 2   timestamp             2937 non-null   object
 3   time_until_update     1 non-null      object
dtypes: object(4)
memory usage: 91.9+ KB


In [4]:
# Зміна типів даних
df['timestamp'] = pd.to_numeric(df['timestamp'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit = 'ns')
df['value'] = df['value'].astype(int)

In [5]:
df.tail(5)

,value,value_classification,timestamp,time_until_update
2932,11,Extreme Fear,1970-01-01 00:00:01.517788800,NaN
2933,24,Extreme Fear,1970-01-01 00:00:01.517702400,NaN
2934,40,Fear,1970-01-01 00:00:01.517616000,NaN
2935,15,Extreme Fear,1970-01-01 00:00:01.517529600,NaN
2936,30,Fear,1970-01-01 00:00:01.517443200,NaN


In [6]:
# Завантаження даних з цінами на BTC
btc_finance = yf.download ('BTC-USD', start = '2018-02-05')
btc_finance.head()

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
Date,,,,,
2018-02-05,6955.270020,8364.839844,6756.680176,8270.540039,9285289984
2018-02-06,7754.000000,7850.700195,6048.259766,7051.750000,13999800320
2018-02-07,7621.299805,8509.110352,7236.790039,7755.490234,9169280000
2018-02-08,8265.589844,8558.769531,7637.859863,7637.859863,9346750464
2018-02-09,8736.980469,8736.980469,7884.709961,8271.839844,6784820224


In [7]:
# Форматування даних щоб отримати колнки які потрібні для аналізу
btc = btc_finance.reset_index()

In [8]:
btc.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']

In [9]:
btc.head()

,Date,Close,High,Low,Open,Volume
0,2018-02-05,6955.270020,8364.839844,6756.680176,8270.540039,9285289984
1,2018-02-06,7754.000000,7850.700195,6048.259766,7051.750000,13999800320
2,2018-02-07,7621.299805,8509.110352,7236.790039,7755.490234,9169280000
3,2018-02-08,8265.589844,8558.769531,7637.859863,7637.859863,9346750464
4,2018-02-09,8736.980469,8736.980469,7884.709961,8271.839844,6784820224


In [10]:
# Перевірка типів даних
btc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2937 entries, 0 to 2936
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    2937 non-null   datetime64[ns]
 1   Close   2937 non-null   float64       
 2   High    2937 non-null   float64       
 3   Low     2937 non-null   float64       
 4   Open    2937 non-null   float64       
 5   Volume  2937 non-null   int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 137.8 KB


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2937 entries, 0 to 2936
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   value                 2937 non-null   int64         
 1   value_classification  2937 non-null   object        
 2   timestamp             2937 non-null   datetime64[ns]
 3   time_until_update     1 non-null      object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 91.9+ KB


In [12]:
# Об'єднання даних в одну DataFrame
df_final = pd.merge(df, btc, left_on = 'timestamp', right_on = 'Date')
df_final.head()

,value,value_classification,timestamp,time_until_update,Date,Close,High,Low,Open,Volume


In [13]:
# Видалення зайвої колонки
df_final = df_final.drop(columns = ['time_until_update', 'Date'])

In [14]:
df_final.columns = df_final.columns.str.lower()
df_final.head()

,value,value_classification,timestamp,close,high,low,open,volume


In [15]:
# Загальний обрахунок для огляду
analysis = df_final.groupby('value_classification')['close'].mean().sort_values()
analysis

Series([], Name: close, dtype: float64)

In [16]:
# Збереження фінального файлу
df_final.to_excel(r'C:\Users\PC\Desktop\google trends\btc_analysis.xlsx', index=False)